<a href="https://colab.research.google.com/github/Itsuki-Hamano123/auto_ml/blob/master/TPOT/blood_transfusion/tpot_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -q tpot
%pip show tpot scikit-learn

     |████████████████████████████████| 92kB 2.5MB/s 
     |████████████████████████████████| 163kB 9.0MB/s 
Name: TPOT
Version: 0.11.6
Summary: Tree-based Pipeline Optimization Tool
Home-page: https://github.com/EpistasisLab/tpot
Author: Randal S. Olson
Author-email: rso@randalolson.com
License: GNU/LGPLv3
Location: /usr/local/lib/python3.6/dist-packages
Requires: update-checker, stopit, scipy, numpy, deap, joblib, tqdm, scikit-learn, pandas
Required-by: 
---
Name: scikit-learn
Version: 0.22.2.post1
Summary: A set of python modules for machine learning and data mining
Home-page: http://scikit-learn.org
Author: None
Author-email: None
License: new BSD
Location: /usr/local/lib/python3.6/dist-packages
Requires: joblib, numpy, scipy
Required-by: yellowbrick, umap-learn, TPOT, textgenrnn, sklearn, sklearn-pandas, mlxtend, lucid, lightgbm, librosa, imbalanced-learn, fancyimpute


[輸血データセット:https://www.openml.org/d/1464](https://www.openml.org/d/1464)

In [47]:
import cloudpickle, datetime, os

import numpy as np

from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tpot import TPOTClassifier

# データ読み込み

In [33]:
%%time
DATA_ID = '1464' #@param{type:'string'}

# データセットをフェッチ
X, y = fetch_openml(data_id=DATA_ID, return_X_y=True, as_frame=True)

def processing_y(y):
    '''ラベルyの加工'''
    def _convert_dtype(type='int64'):
        '''dtype: category(文字列型)をint64に変更'''
        return y.astype('int64')
    
    def _indicator_start_0():
        '''インディケータ開始の数を0にする'''
        return y - y[np.argmin(y)]

    y = _convert_dtype()
    y = _indicator_start_0()
    return y

y = processing_y(y)
display(X)
display(y)


SEED = 7 #@param{type:'number'}
SPLIT = 0.2 #@param{type:'number'}

train_X, test_X, train_y, test_y = train_test_split(X, y,
                                                    test_size=SPLIT,
                                                    random_state=SEED,
                                                    shuffle=True)
display('train shape:{}'.format(train_X.shape))
display('test shape:{}'.format(test_X.shape))

,V1,V2,V3,V4
0,2.0,50.0,12500.0,98.0
1,0.0,13.0,3250.0,28.0
2,1.0,16.0,4000.0,35.0
3,2.0,20.0,5000.0,45.0
4,1.0,24.0,6000.0,77.0
...,...,...,...,...
743,23.0,2.0,500.0,38.0
744,21.0,2.0,500.0,52.0
745,23.0,3.0,750.0,62.0
746,39.0,1.0,250.0,39.0


0      1
1      1
2      1
3      1
4      0
      ..
743    0
744    0
745    0
746    0
747    0
Name: Class, Length: 748, dtype: int64

'train shape:(598, 4)'

'test shape:(150, 4)'

CPU times: user 42.9 ms, sys: 3.23 ms, total: 46.1 ms
Wall time: 46.1 ms


# パイプライン構築

In [34]:
GENERATION =  3#@param{type:'number'}
POPULATION =  50#@param{type:'number'}
CV =  5#@param{type:'number'}
EARLY_STOP_ROUND = 2 #@param:{type:'number'}
N_JOBS = -1 #@param{type:'number'}
VERBOSITY = 2 #@param{type:'number'}

SEARCH_STRATEGY = None

tpot = TPOTClassifier(config_dict=SEARCH_STRATEGY,
                      generations=GENERATION, # (default:100)
                      population_size=POPULATION, # (default:100)
                      cv=CV, #(default:5)
                      #early_stop=EARLY_STOP_ROUND,
                      n_jobs=N_JOBS, # (default:1)
                      random_state=SEED,
                      verbosity=VERBOSITY,
                      warm_start=True, # 以前のfit結果があれば続きから開始
                      )


In [35]:
%%time
tpot.fit(features=train_X, target=train_y)


Generation 1 - Current best internal CV score: 0.804327731092437

Generation 2 - Current best internal CV score: 0.8059943977591036

Generation 3 - Current best internal CV score: 0.8059943977591036

Generation 4 - Current best internal CV score: 0.8059943977591036

Generation 5 - Current best internal CV score: 0.8076890756302519

Best pipeline: XGBClassifier(CombineDFs(ExtraTreesClassifier(input_matrix, bootstrap=True, criterion=entropy, max_features=0.55, min_samples_leaf=15, min_samples_split=20, n_estimators=100), input_matrix), learning_rate=0.001, max_depth=3, min_child_weight=11, n_estimators=100, nthread=1, subsample=0.7500000000000001)
CPU times: user 22.5 s, sys: 1.11 s, total: 23.6 s
Wall time: 2min 33s


TPOTClassifier(config_dict=None, crossover_rate=0.1, cv=5,
               disable_update_check=False, early_stop=None, generations=5,
               log_file=None, max_eval_time_mins=5, max_time_mins=None,
               memory=None, mutation_rate=0.9, n_jobs=-1, offspring_size=None,
               periodic_checkpoint_folder=None, population_size=50,
               random_state=7, scoring=None, subsample=1.0, template=None,
               use_dask=False, verbosity=2, warm_start=True)

In [45]:
# 探索されたパイプライン表示
#display(tpot.pareto_front_fitted_pipelines_)

# ベストなパイプライン表示
display(tpot.fitted_pipeline_)

Pipeline(memory=None,
         steps=[('featureunion',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('stackingestimator',
                                                 StackingEstimator(estimator=ExtraTreesClassifier(bootstrap=True,
                                                                                                  ccp_alpha=0.0,
                                                                                                  class_weight=None,
                                                                                                  criterion='entropy',
                                                                                                  max_depth=None,
                                                                                                  max_features=0.55,
                                                                                                  max_leaf_nodes=None,
                   

#パイプラインの出力

In [36]:
%cd /content/drive/My Drive/機械学習練習/AutoML/TPOT
%ls

/content/drive/My Drive/機械学習練習/AutoML/TPOT
blood-transfusion_classification.ipynb  radon-regressor_drop_country.ipynb
export_pipeline/                        radon-regressor.ipynb


In [37]:
%%time

EXPORT_DIR =  './export_pipeline' # @param {type:'string'}
file_prefix = 'blood-transfusion_classification_pipeline' # @param{type:'string'}

now = datetime.datetime.now().strftime('%Y%m%d_%H%m%s')
export_file = os.path.join(EXPORT_DIR, now+'_'+file_prefix)

# ベストなパイプラインのスクリプトファイル出力
tpot.export(output_file_name=export_file+'.py')


def dump_pkl(obj, path):
    '''objをpklファイルで出力'''
    with open(path, 'wb') as f:
        f.write(cloudpickle.dumps(obj))

# ベストなパイプラインをpklで出力
best_pipline = tpot.fitted_pipeline_
dump_pkl(best_pipline, export_file+'.pkl')

%ls {EXPORT_DIR}

20201026_08101603700167_radon-regressor_drop_country_pipeline.pkl
20201026_08101603700167_radon-regressor_drop_country_pipeline.py
20201026_08101603700544_radon-regressor_pipeline.pkl
20201026_08101603700544_radon-regressor_pipeline.py
20201026_08101603701198_radon-regressor_pipeline.pkl
20201026_08101603701198_radon-regressor_pipeline.py
20201027_00101603759647_radon-regressor_drop_country_pipeline.pkl
20201027_00101603759647_radon-regressor_drop_country_pipeline.py
20201027_01101603761063_radon-regressor_drop_country_pipeline.pkl
20201027_01101603761063_radon-regressor_drop_country_pipeline.py
20201027_01101603762717_radon-regressor_drop_country_pipeline.pkl
20201027_01101603762717_radon-regressor_drop_country_pipeline.py
20201027_02101603767263_blood-transfusion_classification_pipeline.pkl
20201027_02101603767263_blood-transfusion_classification_pipeline.py
20201028_05101603862054_blood-transfusion_classification_pipeline.pkl
20201028_05101603862054_blood-transfusion_classification_

# モデルの評価

## TPOTの組み込み評価関数

In [46]:
score = tpot.score(testing_features=test_X, testing_target=test_y)
display(score)

0.7466666666666667

## sklearnの評価関数

In [52]:
# 予測結果の取得
train_y_pred = tpot.predict(features=train_X)
test_y_pred = tpot.predict(features=test_X)

train_report = classification_report(y_true=train_y, y_pred=train_y_pred)
test_report = classification_report(y_true=test_y, y_pred=test_y_pred)

print(train_report)
print('----- -----')
print(test_report)



              precision    recall  f1-score   support

           0       0.86      0.91      0.88       458
           1       0.64      0.49      0.56       140

    accuracy                           0.82       598
   macro avg       0.75      0.70      0.72       598
weighted avg       0.80      0.82      0.81       598

----- -----
              precision    recall  f1-score   support

           0       0.78      0.91      0.84       112
           1       0.50      0.26      0.34        38

    accuracy                           0.75       150
   macro avg       0.64      0.59      0.59       150
weighted avg       0.71      0.75      0.72       150

